In [ ]:
import numpy as np
import scipy.special as sc
import matplotlib.pyplot as plt
from scipy.io import loadmat
from ipywidgets import interact, interactive, widgets

from scipy.constants import h, c, k, e, m_e, epsilon_0,physical_constants

# k_ev = physical_constants['Boltzmann constant in eV/K'][0]
sigma_sb = physical_constants['Stefan-Boltzmann constant'][0]
L = 2.44e-8

#use the same constant as in rswarp
k_ev = 8.6173324e-5
from rswarp.cathode import sources

## _Define functions_

In [ ]:
def rd_current(phi, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 1.20e6#4 * np.pi * m_e * k**2 * e / h**3
    
    return A * T**2 * np.exp(-1.*phi / (k_ev * T))

def Vretard(J,phi_c, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 1.20e6#4 * np.pi * m_e * k**2 * e / h**3
    
    return -np.log(J/(A * T**2)) *  (k_ev * T)-phi_c

In [ ]:
def xiPara(gamma, midpoint=0.00001, nn=1e6):
    if gamma==0:
        return 0, 0
    nn=int(nn)
    if gamma<=midpoint:
        gamma_arr = np.linspace(0,gamma,nn)
    else:
        gamma_arr = np.concatenate((np.linspace(0,midpoint,nn,endpoint=False),np.linspace(midpoint,gamma,nn)))
    gamma_arr[0] = 1e-14
    xim = np.zeros_like(gamma_arr)
    xip = np.zeros_like(gamma_arr)
    xim = -np.trapz(1 / np.sqrt(np.exp(gamma_arr) - 1 + np.exp(gamma_arr) * sc.erf(np.sqrt(gamma_arr))
                             - 2 * np.sqrt(gamma_arr/np.pi)),gamma_arr)
    xip = np.trapz(1 / np.sqrt(np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi)),gamma_arr)
    return xim, xip

def xiPara_sr(gamma, srefprob, midpoint=0.00001, nn=1e6):
    if srefprob == 0: return xiPara(gamma, midpoint, nn)
    if gamma==0:
        return 0, 0
    nn=int(nn)
    if gamma<=midpoint:
        gamma_arr = np.linspace(0,gamma,nn)
    else:
        gamma_arr = np.concatenate((np.linspace(0,midpoint,nn,endpoint=False),np.linspace(midpoint,gamma,nn)))
    gamma_arr[0] = 1e-14
    xim = -np.trapz(1 / np.sqrt(np.exp(gamma_arr) - 1 + np.exp(gamma_arr) * sc.erf(np.sqrt(gamma_arr))
                             - 2 * np.sqrt(gamma_arr/np.pi) + srefprob * (np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi))),gamma_arr)
    xip = np.trapz(1 / np.sqrt(np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi) + srefprob * (np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi))),gamma_arr)
    return xim, xip


def xiPara_dr(gamma, drefprob, midpoint=0.00001, nn=1e6):
    if drefprob == 0: return xiPara(gamma, midpoint, nn)
    if gamma==0:
        return 0, 0
    nn=int(nn)
    if gamma<=midpoint:
        gamma_arr = np.linspace(0,gamma,nn)
    else:
        gamma_arr = np.concatenate((np.linspace(0,midpoint,nn,endpoint=False),np.linspace(midpoint,gamma,nn)))
    gamma_arr[0] = 1e-14
    xim = -np.trapz(1 / np.sqrt(np.exp(gamma_arr) - 1 + np.exp(gamma_arr) * sc.erf(np.sqrt(gamma_arr))
                             - 2 * np.sqrt(gamma_arr/np.pi) + drefprob * (np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi))),gamma_arr)#
    xip = np.trapz(1 / np.sqrt(np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi) + drefprob * (np.exp(gamma_arr) * sc.erfc(np.sqrt(gamma_arr)) - 1
                             + 2 * np.sqrt(gamma_arr/np.pi))),gamma_arr)#
    return xim, xip

In [ ]:
def x0_fun(J,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    return x0

def xiFromJ(J,x,xm,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    xi = (x-xm)/x0
    return xi

def JFromxi(xi,x,xm,T):
    x0 = (x-xm)/xi
    J = ((epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/x0)**2
    
    return J

def calculate_currfraction(T, V_gap, drefprob, E_arr=None):
    if E_arr is None:
        if V_gap > 0:
            frac= (1-drefprob) * (1 + drefprob * e * V_gap / k / T + drefprob * (1 - drefprob) * (e * V_gap / k / T) ** 2 * np.exp((1 - drefprob) * e * V_gap / k / T ) * sc.expi(-(1 - drefprob) * e * V_gap / k / T))
        else:
            frac = (1-drefprob)   
    else:
        if len(drefprob) != len(E_arr):
            raise RuntimeError("energy array (E_arr) and diffusive reflectivity (drefprob) must be in the same size")
        frac = np.trapz(np.exp(-E_arr / k / T) * (1 - drefprob) / (1 - drefprob * e * V_gap / (e * V_gap + E_arr)) * E_arr / (k * T) ** 2, E_arr)
    
    return frac

## _Construct gamma-xi table_ (Do not run this section, the gamma-xi tables are already computed in Mathematica and saved as csv files)

In [ ]:
npoints = int(1e3)
gamma1 = np.linspace(0,5,npoints,endpoint = False)
gamma2 = 0.5*10**(np.linspace(1, 3, npoints))
gammas = np.concatenate((gamma1,gamma2))

In [ ]:
xim = np.zeros_like(gammas)
xip = np.zeros_like(gammas)
for i, gamma in enumerate(gammas):
    xim[i],xip[i] = xiPara(gamma)
np.save('gamma_xi_table.npy', (gammas,xim, xip))

In [ ]:
srefprobs = [0.5]#0.1,0.2,0.3,0.4,0.6,0.7,0.8,0.9
for srefprob in srefprobs:
    xim = np.zeros_like(gammas)
    xip = np.zeros_like(gammas)
    for i, gamma in enumerate(gammas):
        xim[i],xip[i] = xiPara_sr(gamma, srefprob)
    np.save('gamma_xi_table_sr'+str(srefprob)+'.npy', (gammas,xim, xip))


In [ ]:
drefprobs = [0.5]#0.1,0.2,0.3,0.4,0.6,0.7,0.8,0.9
for drefprob in drefprobs:
    xim = np.zeros_like(gammas)
    xip = np.zeros_like(gammas) 
    for i, gamma in enumerate(gammas):
        xim[i],xip[i] = xiPara_dr(gamma, drefprob)
    np.save('gamma_xi_table_dr'+str(drefprob)+'.npy', (gammas,xim, xip))


## _Start i-v calculations_

In [ ]:
srefprob = 0.5
drefprob = 0.5
srefprob_str = str(srefprob).replace('.','')
drefprob_str = str(drefprob).replace('.','')
#Load gamma-xi table calculated via Nintegrate in Mathematica (with wider range of gamma) or via trapz in Python
data_source = 'mathematica' #'python'
if data_source == 'mathematica':
    data = np.genfromtxt('./data_table.csv',delimiter=',')#_0to100highres
    gammas = data[:,0]
    xim = np.real(data[:,1])
    xip = np.real(data[:,2])
    data = np.genfromtxt('./data_table_dr'+str(drefprob)+'.csv',delimiter=',')
    gammas_dr = data[:,0]
    xim_dr = data[:,1]
    xip_dr = data[:,2]
    data = np.genfromtxt('./data_table_sr'+str(srefprob)+'.csv',delimiter=',')
    gammas_sr = data[:,0]
    xim_sr = data[:,1]
    xip_sr = data[:,2]
elif data_source == 'python':
    gammas,xim,xip = np.load('gamma_xi_table.npy')
    gammas_sr,xim_sr,xip_sr = np.load('gamma_xi_table_sr'+str(srefprob)+'.npy')
    gammas_dr,xim_dr,xip_dr = np.load('gamma_xi_table_dr'+str(drefprob)+'.npy')

In [ ]:
plt.plot(xim,gammas)
plt.plot(xip,gammas)
plt.plot(xim_dr,gammas_dr)
plt.plot(xip_dr,gammas_dr)
plt.plot(xim_sr,gammas_sr)
plt.plot(xip_sr,gammas_sr)
plt.xlim([-5,140])
plt.ylim([0,500])
plt.xlabel('\ksi')
plt.ylabel('\gamma')

In [ ]:
# parameters:
phi_e = 2.174#2.3#2.22 #eV
phi_c = 0.381#1.73#2.1 #eV 1.73
T_e =  1414 + 273.15 #K 1050 1000
dgap = '10'
d = float(dgap)*1e-6 #m 3.16

In [ ]:
# saturation point:
J_sat = sources.j_rd(T_e, phi_e)#rd_current(phi_e,T_e)
xi_e = xiFromJ(J_sat,d,0,T_e)
gamma_e = np.interp(xi_e,xip,gammas)
V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
print("V_sat:",V_sat)

# first guess of the critical point:
J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
gamma_E = np.log(J_sat/J_c_0)
V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

# critical point:
J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
gamma_e_arr = np.log(J_sat/J_arr)
excludeind = np.where(gamma_e_arr<0)
np.put(gamma_e_arr,excludeind,0)
xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
np.put(xi_c_arr,excludeind,-1)
xi_c = min(xi_c_arr[xi_c_arr>=0])
J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
V_r = Vretard(J_r, phi_c, T_e)
print("V_cr:", V_r)

# space charge limited iv:
n_scl = 10000000
J_scl = np.linspace(J_sat,J_r,n_scl)
gamma_e_scl = np.log(J_sat/J_scl)
xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e

# construct the entire i-v table:
n_sat = 100
n_r = 100

J_sat_arr = np.full(n_sat, J_sat)
V_sat_arr = np.linspace(V_sat-4,V_sat,n_sat)
V_r_arr = np.linspace(V_r,V_r+ 2, n_r)
J_r_arr = rd_current(phi_c+V_r_arr, T_e)

V_data = np.concatenate((V_sat_arr,V_scl,V_r_arr))#
J_data = np.concatenate((J_sat_arr,J_scl,J_r_arr))#

## _Specular reflection_

In [ ]:
# saturation point:
J_sat = rd_current(phi_e,T_e)
xi_e = xiFromJ(J_sat,d,0,T_e)
gamma_e_sr = np.interp(xi_e,xip_sr,gammas_sr)
# gamma_e = 10
V_sat_sr = phi_e-phi_c-gamma_e_sr*k_ev*T_e
print("V_sat_sr is:",V_sat_sr)

# first guess of the critical point:
J_c_0_sr = JFromxi(max(np.abs(xim_sr)),0,d,T_e)
gamma_E_sr = np.log(J_sat/J_c_0_sr)
V_c_0_sr = -(gamma_E_sr*k*T_e/e-phi_e+phi_c)

# critical point:
J_arr_sr = np.exp(np.linspace(-6,3,10000))*J_c_0_sr
gamma_e_arr_sr = np.log(J_sat/J_arr_sr)
excludeind = np.where(gamma_e_arr_sr<0)
np.put(gamma_e_arr_sr,excludeind,0)
xi_e_arr_sr = np.interp(gamma_e_arr_sr, gammas_sr,xim_sr)
xi_c_arr_sr = d/x0_fun(J_arr_sr, T_e) + xi_e_arr_sr
np.put(xi_c_arr_sr,excludeind,-1)
xi_c_sr = min(xi_c_arr_sr[xi_c_arr_sr>=0])
J_r_sr = J_arr_sr[np.where(xi_c_arr_sr==xi_c_sr)[0][0]]
V_r_sr = Vretard(J_r_sr, phi_c, T_e)
print("V_cr_sr", V_r_sr)

# space charge limited iv:

J_scl_sr = np.linspace(J_sat,J_r_sr,n_scl)
gamma_e_scl_sr = np.log(J_sat/J_scl_sr)
xi_e_scl_sr = np.interp(gamma_e_scl_sr, gammas_sr,xim_sr)
xi_c_scl_sr = d/x0_fun(J_scl_sr, T_e) + xi_e_scl_sr
gamma_c_scl_sr = np.interp(xi_c_scl_sr,xip_sr,gammas_sr)
V_scl_sr = phi_e-phi_c+(gamma_e_scl_sr-gamma_c_scl_sr)*k*T_e/e

# construct the entire i-v table:
J_sat_arr_sr = np.full(n_sat, J_sat)
V_sat_arr_sr = np.linspace(V_sat_sr-4,V_sat_sr,n_sat)
V_r_arr_sr = np.linspace(V_r_sr,V_r_sr+ 2, n_r)
J_r_arr_sr = rd_current(phi_c+V_r_arr_sr, T_e)

V_data_sr = np.concatenate((V_sat_arr_sr,V_scl_sr,V_r_arr_sr))#
J_data_sr = np.concatenate((J_sat_arr_sr,J_scl_sr,J_r_arr_sr)) * (1 - srefprob)#
V_a_sr = np.concatenate((phi_e-phi_c-V_sat_arr_sr,gamma_c_scl_sr*k_ev*T_e,np.zeros_like(V_r_arr_sr)))

## _Diffuse reflection_

In [ ]:
# saturation point:
J_sat = rd_current(phi_e,T_e)
xi_e = xiFromJ(J_sat,d,0,T_e)
gamma_e_dr = np.interp(xi_e,xip_dr,gammas_dr)
# gamma_e = 10
V_sat_dr = phi_e-phi_c-gamma_e_dr*k_ev*T_e
print("V_sat_dr is:",V_sat_dr)

# first guess of the critical point:
J_c_0_dr = JFromxi(max(np.abs(xim_dr)),0,d,T_e)
gamma_E_dr = np.log(J_sat/J_c_0_dr)
V_c_0_dr = -(gamma_E_dr*k*T_e/e-phi_e+phi_c)

# critical point:
J_arr_dr = np.exp(np.linspace(-6,3,10000))*J_c_0_dr
gamma_e_arr_dr = np.log(J_sat/J_arr_dr)
excludeind = np.where(gamma_e_arr_dr<0)
np.put(gamma_e_arr_dr,excludeind,0)
xi_e_arr_dr = np.interp(gamma_e_arr_dr, gammas_dr,xim_dr)
xi_c_arr_dr = d/x0_fun(J_arr_dr, T_e) + xi_e_arr_dr
np.put(xi_c_arr_dr,excludeind,-1)
xi_c_dr = min(xi_c_arr_dr[xi_c_arr_dr>=0])
J_r_dr = J_arr_dr[np.where(xi_c_arr_dr==xi_c_dr)[0][0]]
V_r_dr = Vretard(J_r_dr, phi_c, T_e)
print("V_cr_dr", V_r_dr)

# space charge limited iv:
J_scl_dr = np.linspace(J_sat,J_r_dr,n_scl)
gamma_e_scl_dr = np.log(J_sat/J_scl_dr)
xi_e_scl_dr = np.interp(gamma_e_scl_dr, gammas_dr,xim_dr)
xi_c_scl_dr = d/x0_fun(J_scl_dr, T_e) + xi_e_scl_dr
gamma_c_scl_dr = np.interp(xi_c_scl_dr,xip_dr,gammas_dr)
V_scl_dr = phi_e-phi_c+(gamma_e_scl_dr-gamma_c_scl_dr)*k*T_e/e

# construct the entire i-v table:
J_sat_arr_dr = np.full(n_sat, J_sat)
V_sat_arr_dr = np.linspace(V_sat_dr-4,V_sat_dr,n_sat)
V_r_arr_dr = np.linspace(V_r_dr,V_r_dr+ 2, n_r)
J_r_arr_dr = rd_current(phi_c+V_r_arr_dr, T_e)

V_data_dr = np.concatenate((V_sat_arr_dr,V_scl_dr,V_r_arr_dr))#
J_data_dr = np.concatenate((J_sat_arr_dr,J_scl_dr,J_r_arr_dr))
V_a_dr = np.concatenate((phi_e-phi_c-V_sat_arr_dr,gamma_c_scl_dr*k_ev*T_e,np.zeros_like(V_r_arr_dr)))
for i,V in enumerate(V_a_dr):
    J_data_dr[i] = J_data_dr[i] * calculate_currfraction(T_e, V, 0.5)

## _Plot i-v_

In [ ]:
simData = np.load('reflection/Littau_benchmark/Littau/benchmark_calc/iv_noreflection_'+dgap+'um_1414C.npy')
simData_dr05 = np.load('reflection/Littau_benchmark/Littau/benchmark_calc/iv_noreflection_'+dgap+'um_dr'+drefprob_str+'.npy')
simData_sr05 = np.load('reflection/Littau_benchmark/Littau/benchmark_calc/iv_noreflection_'+dgap+'um_sr'+srefprob_str+'.npy')
simData_old = np.load('reflection/Littau_benchmark/Littau/benchmark_calc/iv_noreflection_'+dgap+'um_1414C_old.npy')

In [ ]:
# plt.plot(np.full(100,(phi_e - phi_c)), np.linspace(0,150,100), '--k',label=r"V_sat")#
plt.plot(V_data, J_data*1e-4, label = 'no reflection')
# plt.plot(V_data_dr, J_data_dr*1e-4, label = 'diffusion reflection 0.5')
# plt.plot(V_data_sr, J_data_sr*1e-4, label = 'specular reflection 0.5')
plt.plot(-simData[0][0], simData[0][1], "o", lw=2, label=r"PIC simulation (no reflection)")
# plt.plot(-simData_dr05[0][0], simData_dr05[0][1], "o", lw=2, label=r"PIC simulation (diffuse reflection 0.5)")
# plt.plot(-simData_sr05[0][0], simData_sr05[0][1], "o", lw=2, label=r"PIC simulation (specular reflection 0.5)")
# plt.plot(-simData_old[0][0], simData_old[0][1], "o", lw=2, label=r"PIC simulation (no reflection) old")

plt.xlim([-13,3])
# plt.ylim([0.0,10])
# plt.xlim([-1.0,1.7])
plt.title(str(round(T_e - 273.15))+'C')
plt.xlabel('applied voltage (V)')
plt.ylabel('current_density (A/cm^2)')
plt.legend(loc="lower left")
# plt.savefig('iv_'+dgap+'um_'+str(round(T_e - 273.15))+'C.png')

In [ ]:
# plt.semilogy(np.full(100, (phi_e - phi_c)), np.linspace(0,1e3,100), '--k')
plt.semilogy(V_data, J_data*1e-4, label = 'no reflection')
# plt.semilogy(V_data_dr, J_data_dr*1e-4, label = 'diffusion reflection 0.5')
# plt.semilogy(V_data_sr, J_data_sr*1e-4, label = 'specular reflection 0.5')
plt.semilogy(-simData[0][0], simData[0][1], "o", lw=2, label=r"PIC simulation (no reflection)")
# plt.semilogy(-simData_dr05[0][0], simData_dr05[0][1], "o", lw=2, label=r"PIC simulation (diffuse reflection)")
# plt.semilogy(-simData_sr05[0][0], simData_sr05[0][1], "o", lw=2, label=r"PIC simulation (specular reflection)")
plt.title(str(round(T_e - 273.15))+'C')
plt.xlabel('applied voltage (V)')
plt.ylabel('current_density (A/cm^2)')
plt.xlim([-13,3])
# plt.ylim([1e-2,1e0])
# plt.ylim([1.5e-2,0.18])
plt.legend()
# plt.savefig('iv_'+dgap+'um_semilogy_'+str(round(T_e - 273.15))+'C.png')